In [133]:
# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afw_display

# Firefly client imports
from firefly_client import FireflyClient

# Standard libraries in support of Firefly display
from urllib.parse import urlparse, urlunparse, ParseResult
from IPython.display import IFrame, display, Markdown
import os

# Identify 2 individual visits that point to the same point on the sky. 
# Use the HSC WIDE dataset 
# /datasets/hsc/repo/rerun/DM-10404/WIDE/
# Cheating can be achieved viu sqlite3 /datasets/hsc/repo/registry.sqlite3 
# select * from raw limit 10
data_dir = '/datasets/hsc/repo/rerun/DM-10404/WIDE/'

In [134]:
# Create a butler from this data directory
from lsst.daf.persistence import Butler
butler = Butler(data_dir)
#print(butler.getUri)
# get all the visits over the SSP_WIDE survey
visitPointings = butler.queryMetadata('calexp',('visit', 'pointing'),{'field':'SSP_WIDE'})
#print(visitPointings)

In [135]:
# Lets try to pull up 2 images from different fields of the same pointing
# 374, 376, pointing :  814
dataId1 = {'visit': 374, 'ccd':42}
dataId2 = {'visit': 376, 'ccd':42}

In [136]:
calexp1 = butler.get('calexp', **dataId1)
calexp2 = butler.get('calexp', **dataId2)

In [137]:
# Create a display
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lspdev.ncsa.illinois.edu'

In [138]:
#ff='{}/firefly/slate.html?__wsch={}'.format(server, my_channel)
#IFrame(ff,800,600)

In [139]:
afwDisplay.setDefaultBackend('firefly')
afw_display_1 = afwDisplay.getDisplay(frame=1, 
                                    name=my_channel)
afw_display_2 = afwDisplay.getDisplay(frame=2, 
                                    name=my_channel)

In [140]:
afw_display_1.mtv(calexp1, "Calexp %d" % dataId1['visit'])
afw_display_2.mtv(calexp2, "Calexp %d" % dataId2['visit'])

In [141]:
# C1 date
c1VisitInfo = calexp1.getInfo().getVisitInfo()
c1Date = c1VisitInfo.getDate()

# C2
c2VisitInfo = calexp2.getInfo().getVisitInfo()
c2Date = c2VisitInfo.getDate()

# Time difference 
timediff = c2Date.toPython() -c1Date.toPython()
timediff.total_seconds()


# Try to identify visits with the same exposure time 
for visit,pointing in visitPointings[:10] : 
    dataId = {'visit': visit, 'ccd':42}
    calexp = butler.get('calexp', **dataId)
    visitInfo = calexp.getInfo().getVisitInfo()
    print(visit, ",", visitInfo.getExposureTime())


374 , 30.0
376 , 200.0
378 , 30.0
380 , 200.0
382 , 30.0
384 , 200.0
386 , 30.0
388 , 200.0
390 , 30.0
392 , 200.0


In [142]:
# Interim hack. Difference the 30 and 200 second images to get. Go back to find t2 overlapping images